In [133]:
from collections import namedtuple, deque
from typing import List, Tuple, Generator, Union

Position = namedtuple("Position", ["x", "y"])
State = namedtuple("State", ["player", "monster", "steps"])


def parse_maze(input_data: str) -> Tuple[int, int, List[str]]:
    lines = input_data.strip().split("\n")
    w, h = map(int, lines[0].split())
    maze = lines[1:]
    return w, h, maze


def move_monster(player: Position, monster: Position, maze: List[str]) -> Position:
    if player.x != monster.x:
        dx = 1 if player.x > monster.x else -1
        if maze[monster.y][monster.x + dx] != '#':
            return monster._replace(x=monster.x + dx)
    if player.y != monster.y:
        dy = 1 if player.y > monster.y else -1
        if maze[monster.y + dy][monster.x] != '#':
            return monster._replace(y=monster.y + dy)
    return monster

# def is_valid_move(position, w, h, maze, monster):

#     if 0 <= position.x < w and 0 <= position.y < h:
#         return maze[position.y][position.x] != '#' and position != monster
#     return False

def find_positions(maze: List[str]) -> Generator[Tuple[str, Position], None, None]:

    for y, row in enumerate(maze):
        for x, char in enumerate(row):
            if char in 'PME':
                yield char, Position(x, y)


def init_maze(maze: List[str]) -> Tuple[Position, Position, Position]:

    player, monster, exit = None, None, None
    for char, position in find_positions(maze):
        if char == 'P':
            player = position
        elif char == 'M':
            monster = position
        elif char == 'E':
            exit = position
        if all([player, monster, exit]):
            break

    return player, monster, exit

def find_exit(maze: List[str], w: int, h: int, player: Position, monster: Position, exit: Position) -> Union[int, str]:

    directions = [Position(0, 1), Position(1, 0), Position(0, -1), Position(-1, 0)]
    q = deque([State(player, monster, 0)])
    visited = set()

    while q:

        current_state = q.popleft()
        player, monster, steps = current_state.player, current_state.monster, current_state.steps

        if player == exit:
            return steps

        if (player, monster) in visited:
            continue
        visited.add((player, monster))

        for d in directions:
            new_player = Position(player.x + d.x, player.y + d.y)

            if not (0 <= new_player.x < w and 0 <= new_player.y < h) or maze[new_player.y][new_player.x] == '#':
                continue

            new_monster = move_monster(new_player, monster, maze)

            if new_player == exit:
                return steps + 1

            if new_player != new_monster and (new_player, new_monster) not in visited:
                q.append(State(new_player, new_monster, steps + 1))

    return "Impossible"



def chase_in_the_maze(input_data: str) -> Union[int, str]:

    w, h, maze = parse_maze(input_data)
    player, monster, exit = init_maze(maze)
    return find_exit(maze, w, h, player, monster, exit)


In [134]:
input_data = [
"""
7 3
.......
E##M##.
.....P.
""",
"""
6 2
#....P
EM....
"""]
for maze in input_data:
  print(f"Input: {maze}Output:{chase_in_the_maze(maze)}\n")

Input: 
7 3
.......
E##M##.
.....P.
Output:10

Input: 
6 2
#....P
EM....
Output:Impossible



In [135]:
input_data2 = [
"""
5 5
....E
.#.#.
.#M#.
.#.#.
P....
""",

"""
6 6
...E..
......
..##..
..M#..
P##...
......
""",
"""
4 4
E..#
..#M
#..#
P...
""" ,
"""
7 7
E......
.......
.......
..M....
.......
.......
......P
"""
]
for maze in input_data2:
  print(f"Input: {maze}Output:{chase_in_the_maze(maze)}\n")

Input: 
5 5
....E
.#.#.
.#M#.
.#.#.
P....
Output:10

Input: 
6 6
...E..
......
..##..
..M#..
P##...
......
Output:7

Input: 
4 4
E..#
..#M
#..#
P...
Output:5

Input: 
7 7
E......
.......
.......
..M....
.......
.......
......P
Output:12

